In [1]:
import instructor

from openai import OpenAI
from pydantic import BaseModel

In [77]:
from typing_extensions import Annotated
from pydantic import BaseModel, Field, field_validator, ValidationInfo
import pandas as pd

Probability = Annotated[float, Field(strict=True, ge=0, le=1)]


class Citation(BaseModel):
    quote: str = Field(..., min_length=3)
    score: Probability
    author: str = Field(None, min_length=3, max_length=255)
    
    @field_validator('quote')
    @classmethod
    def citation_exists(cls, v: str, info: ValidationInfo):  
        context = info.context
        if context:
            context = context.get('article_n_meta')
            quote_in_article = any({v in elem for elem in context})
            if not quote_in_article:
                raise ValueError(f"Citation `{v}` not found in article_n_meta")
        return v

In [79]:
class Sentiment(BaseModel):
    name: str
    score: float | None
    citations: list[Citation]

    @field_validator("citations")
    @classmethod
    def check_score_iff_citations(cls, citations, values):
        score = values.data["score"]
        no_score_no_citations = score is None and not citations
        score_and_citations = score is not None and citations
        
        if no_score_no_citations or score_and_citations:
            return citations
        
        raise ValueError("Either both a score and citations are present\
            or neither are.")
    
    @field_validator("name")
    @classmethod
    def validate_politicians(cls, name: str, info: ValidationInfo):
        context = info.context
        if context:
            context = context.get('article_n_meta')
            politican_in_article = any({name in elem for elem in context})
            if not politican_in_article:
                raise ValueError(f"Citation `{name}` not found in article_n_meta")
        return name

In [91]:
Sentiment.model_validate(
    {"name": "Pedro Nuno Santos", "score": .5, "citations": [cite]},
    context={"article_n_meta": ["Pedro Nuno Santos"]}
)

Sentiment(name='Pedro Nuno Santos', score=0.5, citations=[Citation(quote='the quick brown fox jumps over the lazy dog', score=0.5, author='John Doe')])

In [93]:
from elections import constants

article = "Mariana Mortágua falou com Vasco da gama sobre a inflação. \
    Enquanto Rui Tavares explicava o sentido da vida a Pedro Nuno Santos"

", ".join([politician for politician in constants.POLITICIANS if politician in article])


'Pedro Nuno Santos, Mariana Mortágua, Rui Tavares'

Issue with loggin in VScode: solve by applying 
https://stackoverflow.com/questions/63872967/duplicate-log-lines-in-vscode-when-debugging-asp-net-core-app

In [6]:
import logging

name = "uibuybyu"
logger = logging.getLogger(name)
ch: logging.Handler = logging.StreamHandler()
logger.addHandler(ch)

    
logger.info("This is a test inuygyubuyvtyhytv")
logger.warning("This is an warning iubtyiugvyt")


This is an warning iubtyiugvyt
This is an warning iubtyiugvyt
This is an warning iubtyiugvyt
This is an warning iubtyiugvyt
This is an warning iubtyiugvyt
This is an warning iubtyiugvyt
